In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
dataset = pd.read_csv("./MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
feature_list = dataset.columns.values

In [5]:
dataset.shape  
dataset = dataset.replace(np.inf,np.nan)    # replacing inf with nan
dataset = dataset.fillna(dataset.mean(numeric_only=True)) # ghen converting nan to mean values

In [6]:
X = dataset.iloc[:,:-1].values
print("shape of X",X.shape)

Y = dataset.iloc[:,-1].values
print("shape of Y",Y.shape)

shape of X (225745, 78)
shape of Y (225745,)


In [7]:
print("is NaN present:",np.any(np.isnan(X)))  # to check whether the array contains nan
print("is inf present:",np.any(np.isinf(X)))  # to check whether the array contains inf
X[X < 0] = 0   # to replace all negative values with zeros

is NaN present: False
is inf present: False


In [8]:
from sklearn.feature_selection import SelectKBest  # feature selection
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

In [9]:
bestfeatures = SelectKBest(score_func = mutual_info_classif, k=50)
fit = bestfeatures.fit(X,Y)
#create df for scores
dfscores = pd.DataFrame(fit.scores_)
#create df for column names
dfcolumns = pd.DataFrame(feature_list)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#naming the dataframe columns
featureScores.columns = ['Selected_columns','Score_chi2'] 
#print 50 best features
print(featureScores.nlargest(50,'Score_chi2')) 

                Selected_columns  Score_chi2
4    Total Length of Fwd Packets    0.651850
63             Subflow Fwd Bytes    0.651391
52           Average Packet Size    0.563188
65             Subflow Bwd Bytes    0.549162
5    Total Length of Bwd Packets    0.548792
12        Bwd Packet Length Mean    0.544830
54          Avg Bwd Segment Size    0.544725
55           Fwd Header Length.1    0.543534
34             Fwd Header Length    0.543255
0               Destination Port    0.537737
10         Bwd Packet Length Max    0.529677
66        Init_Win_bytes_forward    0.498712
8         Fwd Packet Length Mean    0.489277
53          Avg Fwd Segment Size    0.488979
6          Fwd Packet Length Max    0.486598
35             Bwd Header Length    0.476373
23                   Fwd IAT Max    0.460222
20                 Fwd IAT Total    0.457513
21                  Fwd IAT Mean    0.446819
62           Subflow Fwd Packets    0.410248
2              Total Fwd Packets    0.410178
22        